In [1]:
using Pkg;
Pkg.activate("..");
Pkg.instantiate();

  Activating project at `~/dev/uni/amo-individual-project`


In [2]:
import CSV

In [3]:
include("../src/ieee_parser.jl")
include("../src/simple_lp.jl")
include("../src/admm.jl")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-19


print_iteration (generic function with 1 method)

In [4]:
ieee_files = filter(contains("ieee"), readdir("../data"; join=true))
# ieee_files = Vector{String}(["../data/ieee14cdf.txt"])

1-element Vector{String}:
 "../data/ieee14cdf.txt"

In [5]:
import Dates
using DataFrames

function benchmark(files::Vector{String})::DataFrame

	results_df = DataFrame(
		:file => String[],
		:method => String[],
		:elapsed => Float64[],
		:objective => Float64[],
		:has_converged => Bool[],
	)

	for file in files
		busses, lines = read_IEEE_common_data_format(file)
		for mode in ["ADMM-single-threaded", "ADMM-multi-threaded", "simple-lp"]
			println("Solving $file with $mode")
			start = Dates.now()
			objective = Inf
			elapsed = Inf
			has_converged = false

			if mode == "simple-lp"
				_, _, objective = solve_with_lp(busses, lines)
				has_converged = true
			else 
				config = create_admm_config(
						Dict(
							:ρ => 1.0, 
							:use_multithreading => mode == "ADMM-multi-threaded",
							:silent => true,
						)
					)
				_, _, objective, has_converged = solve_with_admm(busses, lines, config)
			end

			elapsed = (Dates.now() - start).value / 1000
			push!(results_df, (file, mode, elapsed, objective, has_converged))
		end
	end

	return results_df
end

benchmark (generic function with 1 method)

In [6]:
results = benchmark(ieee_files)

Solving ../data/ieee14cdf.txt with ADMM-single-threaded
Solving ../data/ieee14cdf.txt with ADMM-multi-threaded
Solving ../data/ieee14cdf.txt with simple-lp


Row,file,method,elapsed,objective,has_converged
,String,String,Float64,Float64,Bool
1,../data/ieee14cdf.txt,ADMM-single-threaded,10.115,9065.08,true
2,../data/ieee14cdf.txt,ADMM-multi-threaded,3.064,9065.08,true
3,../data/ieee14cdf.txt,simple-lp,0.053,9065.0,true


In [7]:
CSV.write("../results/results.csv", results)

"../results/results.csv"